In [1]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold

In [2]:
#df = pd.read_csv('../task2_data/data2_ex.csv')[['class', 'tweet']]
df = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df['class'] = df['class'].apply(lambda x: '__label__' + str(x))

df = df.iloc[np.random.permutation(len(df))]
df_train, df_test = train_test_split(df , test_size=0.2)

df_train[['class', 'tweet']].to_csv('TRAIN.txt', header=False, index=False, sep="\t")
df_test[['class', 'tweet']].to_csv('TEST.txt', header=False, index=False, sep="\t")
df_train, df_test

(            class                                              tweet
 275    __label__0  inside 60's people needed acidity to create th...
 2447   __label__0  “@kingkerbyy: my throat hurts so much, especia...
 6784   __label__0  i can totally see people naming their kid latu...
 14494  __label__0                                        why tamiflu
 8394   __label__0                eliquis may make you bleed to death
 ...           ...                                                ...
 13702  __label__0  24hrs went from 100-0 lol jack &amp; ginger la...
 4491   __label__1  @furrowedfox cymbalta is especially weird i’ve...
 6648   __label__0  idk what victoza is supposed to do but the com...
 10397  __label__0  @thebpdqueen took geodon during 1st pregnancy ...
 4754   __label__0  "@uberfacts: bananas contain a natural chemica...
 
 [16435 rows x 2 columns],
             class                                              tweet
 16187  __label__0  i need soup, painkillers and a ventolin i

In [3]:
import fasttext

graph = []
maxima, maxima_idx = 0, 0

for epoch in range(15):
    model = fasttext.train_supervised(input='TRAIN.txt', epoch=epoch)
    All, Pos = 0, 0
    for (clas, tweet) in zip(df_test['class'], df_test['tweet']):
        All += 1
        if int(clas[-1]) == int(model.predict(tweet, k=2)[0][0][-1]): 
            #print(int(clas[-1]),int(model.predict(tweet, k=2)[0][0][-1]))
            Pos += 1
    graph.append(Pos/All)
    if Pos/All > maxima: maxima, maxima_idx = Pos/All, epoch

print("the best accuracy:")
print(maxima, f'on the {maxima_idx} epoch')


the best accuracy:
0.9184716476028231 on the 5 epoch


In [4]:
graph = pd.DataFrame(graph)
graph.plot(figsize=(15,5))

In [5]:
model.get_word_vector('shot')

array([ 0.00195665,  0.02112359, -0.00465235, -0.00536614,  0.02717513,
        0.02771458, -0.01600133, -0.00541279,  0.03336415,  0.03271822,
       -0.02329163,  0.02708048,  0.00403432,  0.04167367, -0.00319119,
        0.00165758,  0.00658411, -0.00110669, -0.00489333, -0.03105083,
       -0.01304516,  0.02943118,  0.01084871, -0.02222642, -0.00597523,
       -0.0203521 ,  0.01402978, -0.01767584, -0.01227832,  0.03001377,
       -0.00042087,  0.00827276, -0.03124759, -0.02529068, -0.05435429,
        0.00383348, -0.00233029, -0.02969841, -0.01530723, -0.01727826,
        0.01522406,  0.03464001,  0.020714  ,  0.02246993,  0.02007951,
       -0.0104899 ,  0.02386948, -0.00921245, -0.00887349,  0.01994853,
       -0.04415656,  0.00363868, -0.02829388,  0.01761301, -0.01937871,
       -0.01034384,  0.00396115,  0.00301032,  0.01878952,  0.02140084,
       -0.00506342,  0.02538495,  0.03994378,  0.02090154, -0.02159301,
        0.00186988, -0.03033377,  0.02001515,  0.01724798,  0.01

# Получаем векторизованные представления твитов при помощи построенной модели...

In [6]:
import sys
import time

def avg_sentence_fasttext(sentence):
    set_sentence = sentence.split(' ')
    count = len(set_sentence)
    for n, word in enumerate(set_sentence):
        if n == 0: 
            current_sentense = model.get_word_vector(word)
        else:
            current_sentense += model.get_word_vector(word)
    return current_sentense/count

# можно получить список твитов сразу
def combine_all_tweets(df, coloumn='tweet'):
    all_tweets = []
    for tweet in df['tweet']:
        vec_tweet = avg_sentence_fasttext(tweet)
        all_tweets.append(vec_tweet)
    return np.array(all_tweets)

In [7]:
dff = df
dff

class                                              tweet
8604   __label__0  my step dad just told me i need to take latuda...
17890  __label__0  @jusmissk @drtlaleng @anatome_ darvocet worked...
4549   __label__0             @skdny humira starting to help at all?
17888  __label__0  in the service of meridia #skyrim http://t.co/...
13034  __label__0  @estelle_lucas imodium tablets. always take th...
...           ...                                                ...
17415  __label__0  "is that a nuvaring on your foot?" @heidi_gold...
14276  __label__0                    oxycodone is bae right meow lol
3788   __label__0               i want vyvanse in my life right now.
7286   __label__0  @tmccallahan yup metformin 500mgs x4 a day jan...
7804   __label__0  fuck me i g0t a right sh0ck when i saw my vide...

[20544 rows x 2 columns]

In [8]:
#df = pd.read_csv('../task2_data/data2_ex.csv')[['class', 'tweet']]

for i in range(len(dff['tweet'])):
    dff['tweet'][i] = np.array(avg_sentence_fasttext(dff['tweet'][i]))

In [9]:
# записал в файл
dff.to_csv('../task2_data/data2_fasttext.csv', index=False)
dff

class                                              tweet
8604   __label__0  [-0.00064820127, 0.008340099, -0.0024124484, -...
17890  __label__0  [0.00073093025, -0.0010084326, 0.0021366323, 0...
4549   __label__0  [0.00097038207, 0.0065289647, -0.0046977303, -...
17888  __label__0  [-0.0059357383, 0.028732834, -0.012219287, -0....
13034  __label__0  [-0.00036741156, 0.017712584, -0.0073990156, -...
...           ...                                                ...
17415  __label__0  [0.0011150655, 0.0143798115, -0.0075835353, -0...
14276  __label__0  [0.0035060775, -0.002282875, 0.0012057559, -0....
3788   __label__0  [0.0037786397, -0.013342995, 0.008682513, 0.00...
7286   __label__0  [0.00048135855, 0.00028256123, 0.0035020502, 0...
7804   __label__0  [0.0010437175, 6.555556e-05, 0.0012534849, -8....

[20544 rows x 2 columns]

**немного кривоты.. не стал заниматься проблемами экспорта-импорта, сразу проверяю модель SVM на векторах FastText**

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

Train = dff.iloc[np.random.permutation(len(dff))]

X = Train['tweet']
Y = Train['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

f1=[] # array of f1 scores 
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, Y)
print(skf)


print('Stratified K-Fold')
for train_index, test_index in skf.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    X_train = [np.array(X_train)[i] for i in range(len(X_train))]
    y_train = [np.array(y_train)[i] for i in range(len(y_train))]

    X_test = [np.array(X_test)[i] for i in range(len(X_test))]
    y_test = [np.array(y_test)[i] for i in range(len(y_test))]

    SVM = SVC(kernel='linear')
    SVM.fit(X_train,y_train)

    predictions_SVM = SVM.predict(X_test)
    f1.append(round(f1_score(predictions_SVM, y_test, average='macro')*100,3))
    print("SVM F1 Score -> ", f1[-1])

# avg f1
print(f'F1 average:\t {round(np.array(f1).mean(), 3)} %')
print(f'F1 std:\t {round(np.array(f1).std(), 3)}')
print(f'F1 var:\t {round(np.array(f1).var(), 3)}')
print()

(16435,) (4109,) (16435,) (4109,)
StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
Stratified K-Fold
SVM F1 Score ->  88.414
SVM F1 Score ->  85.678
SVM F1 Score ->  85.732
SVM F1 Score ->  84.375
SVM F1 Score ->  83.598
F1 average:	 85.559 %
F1 std:	 1.64
F1 var:	 2.688



In [11]:
print(classification_report(y_test, predictions_SVM))

              precision    recall  f1-score   support

  __label__0       0.94      0.98      0.96      3572
  __label__1       0.85      0.61      0.71       536

    accuracy                           0.93      4108
   macro avg       0.90      0.80      0.84      4108
weighted avg       0.93      0.93      0.93      4108



**прикол с палкой**

In [12]:
import sys
import time

def spinning_cursor():
    while True:
        for cursor in '|/-\\':
            yield cursor

spinner = spinning_cursor()
for _ in range(50):
    sys.stdout.write(next(spinner))
    sys.stdout.flush()
    time.sleep(0.1)
    sys.stdout.write('\b')

In [13]:
import numpy as np
tweets = df_test['tweet'].to_list()

In [14]:

model = fasttext.train_supervised(input='TRAIN.txt', epoch=maxima_idx)
predictions = model.predict(tweets, k=2)

predictions = [int(pred[0][-1]) for pred in predictions[0]]
tests = [int(test[-1]) for test in df_test['class']]

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

print("SVM Accuracy Score -> ", accuracy_score(predictions, tests)*100)
print("SVM F1 Score -> ", f1_score(predictions, tests)*100)
print("SVM Roc_auc Score -> ", roc_auc_score(predictions, tests)*100)

SVM Accuracy Score ->  91.82282793867121
SVM F1 Score ->  39.78494623655913
SVM Roc_auc Score ->  79.31231790466464
